In [278]:
import tensorflow as tf
import numpy as np

In [279]:
X = np.array([1, 2, 3, 4], dtype=np.float32)
y = np.array([3, 5, 7, 9], dtype=np.float32)

In [280]:
W = tf.Variable(0.0) # 가중치
b = tf.Variable(0.0) # 편향

In [281]:
def linear_model(X):
    return W * X + b

In [282]:
def mean_squared_error(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))  # reduce_mean : 평균, square : 제곱(값을 여러개 리스트로 넣으면 하나하나 제곱해줌)

In [283]:
optimizer = tf.optimizers.SGD(learning_rate=0.01)

In [284]:
# 학습 과정 정의
def train_step(X, y):
    with tf.GradientTape() as tape:
        y_pred = linear_model(X)
        loss = mean_squared_error(y, y_pred)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
    return loss

In [285]:
# 100번 단위로 학습 반복하며 오차 비교해보기
epochs = 1000
for epoch in range(epochs):
    loss = train_step(X, y)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss: {loss.numpy()}")

Epoch 0: Loss: 41.0
Epoch 100: Loss: 0.007531234994530678
Epoch 200: Loss: 0.0041345576755702496
Epoch 300: Loss: 0.002269820310175419
Epoch 400: Loss: 0.0012461235746741295
Epoch 500: Loss: 0.0006841050344519317
Epoch 600: Loss: 0.0003755530924536288
Epoch 700: Loss: 0.0002061743725789711
Epoch 800: Loss: 0.00011318828910589218
Epoch 900: Loss: 6.213805318111554e-05


In [286]:
# 학습된 W, b 출력
print(f"학습된 W: {W.numpy()}")
print(f"학습된 b: {b.numpy()}")

학습된 W: 2.0048611164093018
학습된 b: 0.9857079386711121


In [287]:
# 예측 결과
X_test = np.array([5, 6], dtype=np.float32)
y_test_pred = linear_model(X_test)
print(f"예측 결과: {y_test_pred.numpy()}")

예측 결과: [11.010014 13.014875]


## 다른 평가지표 구현하기
## 평가지표에 따른 손실값 확인 및 시각화(선택) 해보기

### 나는 R스퀘어 해보기로 했다.

In [288]:
def r_squared(y_true, y_pred):
    residual_sum_of_squares = tf.reduce_sum(tf.square(y_true - y_pred))
    total_sum_of_squares = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    r2 = 1 - residual_sum_of_squares / (total_sum_of_squares + tf.keras.backend.epsilon())
    return r2

In [289]:
def train_step(X, y):
    with tf.GradientTape() as tape:
        y_pred = linear_model(X)
        loss = r_squared(y, y_pred)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
    return loss

In [290]:
epochs = 1000
epochlist = []
losslist = []
for epoch in range(epochs):
    loss = train_step(X, y)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss: {loss.numpy()}")
        epochlist.append(epoch)
        losslist.append(loss.numpy())


Epoch 0: Loss: 0.9999932050704956
Epoch 100: Loss: 0.999992311000824
Epoch 200: Loss: 0.9999908208847046
Epoch 300: Loss: 0.9996452331542969
Epoch 400: Loss: 0.7535979747772217
Epoch 500: Loss: -174.9620819091797
Epoch 600: Loss: -125663.546875
Epoch 700: Loss: -89744304.0
Epoch 800: Loss: -64091545600.0
Epoch 900: Loss: -45771432919040.0


In [291]:
# 학습된 W, b 출력
print(f"학습된 W: {W.numpy()}")
print(f"학습된 b: {b.numpy()}")

학습된 W: -132451696.0
학습된 b: -45049728.0


In [292]:
# 예측 결과
X_test = np.array([5, 6], dtype=np.float32)
y_test_pred = linear_model(X_test)
print(f"예측 결과: {y_test_pred.numpy()}")

예측 결과: [-7.073082e+08 -8.397599e+08]


In [293]:
import pandas as pd


lossdf = pd.DataFrame(losslist, columns=['loss'])

In [294]:
epochdf = pd.DataFrame(epochlist, columns=['epoch'])

In [295]:
df = pd.concat([epochdf,lossdf], axis=1)
df

,epoch,loss
0,0,9.999932e-01
1,100,9.999923e-01
2,200,9.999908e-01
3,300,9.996452e-01
4,400,7.535980e-01
5,500,-1.749621e+02
6,600,-1.256635e+05
7,700,-8.974430e+07
8,800,-6.409155e+10
9,900,-4.577143e+13


## 해설 : RMSE 써보기.

In [301]:
import tensorflow as tf
import numpy as np

# 데이터 생성 (y = 2x + 1)
X = np.array([1, 2, 3, 4], dtype=np.float32)
y = np.array([3, 5, 7, 9], dtype=np.float32)

# 가중치, 바이어스 초기화
W = tf.Variable(0.0)
b = tf.Variable(0.0)

def linear_model(X):
    return W * X + b

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

optimizer = tf.optimizers.SGD(learning_rate=0.01)

# 학습 과정 정의
def train_step(X, y):
    with tf.GradientTape() as tape:
        y_pred = linear_model(X)
        loss = rmse(y, y_pred)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
    return loss

# 학습 반복
epochs = 1000
for epoch in range(epochs):
    loss = train_step(X, y)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Loss: {loss.numpy()}")

# 학습된 파라미터 출력
print(f"학습된 W: {W.numpy()}")
print(f"학습된 b: {b.numpy()}")

# 예측 결과 출력
X_test = np.array([5, 6], dtype=np.float32)
y_test_pred = linear_model(X_test)
print(f"예측 결과: {y_test_pred.numpy()}")

Epoch 0: Loss: 6.4031243324279785
Epoch 100: Loss: 0.07388278096914291
Epoch 200: Loss: 0.04142230749130249
Epoch 300: Loss: 0.04143679887056351
Epoch 400: Loss: 0.04144427180290222
Epoch 500: Loss: 0.04144619032740593
Epoch 600: Loss: 0.04144619032740593
Epoch 700: Loss: 0.04144619032740593
Epoch 800: Loss: 0.04144619032740593
Epoch 900: Loss: 0.04144619032740593
학습된 W: 2.0135791301727295
학습된 b: 1.0046178102493286
예측 결과: [11.072514 13.086092]


## 해설 : 교차 엔트로피 검증 해보기 위해 분류모델 만들어보기

In [311]:
y_true = np.array([[0,1,0], [1,0,0], [0,1,0]])
y_pred = np.array([[0.1,0.7,0.2], [0.8,0.1,0.15],[0.2,0.6,0.3]]) # 답은 맞추겠지만, 확률 오차값이 누적될 것이다.

In [307]:
tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred)) # 첫번쨰, 두번째, 세번째를 예측한 확률들과 생기는 오차를 평균낸것

<tf.Tensor: shape=(), dtype=float64, numpy=0.41158148766423003>

In [306]:
tf.reduce_mean(tf.abs(y_true -  y_pred)) # MAE

<tf.Tensor: shape=(), dtype=float64, numpy=0.21666666666666667>

In [308]:
tf.reduce_mean(tf.square(y_true -  y_pred)) # MSE

<tf.Tensor: shape=(), dtype=float64, numpy=0.05583333333333334>

In [309]:
tf.sqrt(tf.reduce_mean(tf.square(y_true -  y_pred))) # RMSE

<tf.Tensor: shape=(), dtype=float64, numpy=0.23629078131263043>